In [18]:
import numpy as np
import pandas as pd
import mne
import torch
import keras
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, Activation, Flatten, Input, DepthwiseConv2D
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, MaxPool2D, Lambda, AveragePooling2D, TimeDistributed, ConvLSTM2D, Reshape, SpatialDropout2D, SeparableConv2D
from keras import regularizers, Model
from keras.constraints import max_norm
from keras.models import Sequential

In [19]:
import os
os.environ["KERAS_BACKEND"] = 'torch'

In [20]:
IGE_file = mne.read_epochs("D:\MNE Data\IGE\shirband.fif", preload=False)
TLE_file = mne.read_epochs("D:\MNE Data\IGE\KeshavarziSakine.fif", preload=False)


Reading D:\MNE Data\IGE\shirband.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
588 matching events found
No baseline correction applied
0 projection items activated
Reading D:\MNE Data\IGE\KeshavarziSakine.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
486 matching events found
No baseline correction applied
0 projection items activated


C:\Users\admin\AppData\Local\Temp\ipykernel_17244\4045462511.py:1: RuntimeWarning: This filename (D:\MNE Data\IGE\shirband.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  IGE_file = mne.read_epochs("D:\MNE Data\IGE\shirband.fif", preload=False)
C:\Users\admin\AppData\Local\Temp\ipykernel_17244\4045462511.py:2: RuntimeWarning: This filename (D:\MNE Data\IGE\KeshavarziSakine.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  TLE_file = mne.read_epochs("D:\MNE Data\IGE\KeshavarziSakine.fif", preload=False)


In [21]:
IGE = torch.Tensor(IGE_file.get_data())
IGE_labels = torch.zeros((IGE.shape[0], 1))

TLE = torch.Tensor(TLE_file.get_data())
TLE_labels = torch.ones((TLE.shape[0], 1))

data = np.append(IGE, TLE, axis=0).reshape(-1, 20, 3000, 1)
labels = np.append(IGE_labels, TLE_labels, axis=0)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15, random_state=42, shuffle=False)

Loading data for 588 events and 3000 original time points ...
Loading data for 486 events and 3000 original time points ...


In [22]:
"""
X_train = torch.Tensor(X_train).cuda()
X_test = torch.Tensor(X_test).cuda()
y_train = torch.Tensor(y_train).cuda()
y_test = torch.Tensor(y_test).cuda()
"""

'\nX_train = torch.Tensor(X_train).cuda()\nX_test = torch.Tensor(X_test).cuda()\ny_train = torch.Tensor(y_train).cuda()\ny_test = torch.Tensor(y_test).cuda()\n'

In [23]:
X_train.shape

(912, 20, 3000, 1)

In [24]:
nb_classes = 2
Chans=20
Samples=3000
dropoutRate=0.5
kernLength=125
F1=7
D=2
F2=7
norm_rate=0.25
dropoutType='Dropout'

In [34]:
input1 = Input(shape=(Chans, Samples, 1))

block1 = Conv2D(F1, (1, kernLength), padding='same',
                    input_shape=(1, Chans, Samples),
                    use_bias=False)(input1)

block1 = BatchNormalization(axis=1)(block1)
block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_constraint=max_norm(1.))(block1)
block1 = BatchNormalization(axis=1)(block1)
block1 = Activation('elu')(block1)
block1 = AveragePooling2D((1, 4))(block1)
block1 = Dropout(dropoutRate)(block1)

block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
block2 = BatchNormalization(axis=1)(block2)
block2 = Activation('elu')(block2)
block2 = AveragePooling2D((1, 8))(block2)
block2 = Dropout(dropoutRate)(block2)

flatten = Flatten(name='flatten')(block2)

dense = Dense(1, name='dense', kernel_constraint=max_norm(norm_rate))(flatten)
softmax = Activation('softmax', name='softmax')(dense)




model = Model(input1, softmax)
model.summary()


c:\Users\admin\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 20, 3000, 1)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 3000, 7)       │        875 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_6           │ (None, 20, 3000, 7)       │         80 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ depthwise_conv2d_2              │ (None, 1, 3000, 14)       │        280 │
│ (DepthwiseConv2D)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_7           │ (None, 1, 3000, 14)       │          4 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_4 (Activation)       │ (None, 1, 3000, 14)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ average_pooling2d_4             │ (None, 1, 750, 14)        │          0 │
│ (AveragePooling2D)              │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 750, 14)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ separable_conv2d_2              │ (None, 1, 750, 7)         │        322 │
│ (SeparableConv2D)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_8           │ (None, 1, 750, 7)         │          4 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_5 (Activation)       │ (None, 1, 750, 7)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ average_pooling2d_5             │ (None, 1, 93, 7)          │          0 │
│ (AveragePooling2D)              │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_5 (Dropout)             │ (None, 1, 93, 7)          │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 651)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 1)                 │        652 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax (Activation)            │ (None, 1)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,217 (8.66 KB)

 Trainable params: 2,173 (8.49 KB)

 Non-trainable params: 44 (176.00 B)

In [26]:
opt = opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt , loss = 'binary_crossentropy')

model.fit(X_train, y_train, epochs=15, batch_size=1024)


Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step - loss: 0.8575
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step - loss: 1.0049
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - loss: 1.1525
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.5212
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 34s 34s/step - loss: 0.4417
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.4299
Epoch 7/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - loss: 0.4464
Epoch 8/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step - loss: 0.4510
Epoch 9/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 34s 34s/step - loss: 0.4398
Epoch 10/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - loss: 0.4145
Epoch 11/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - loss: 0.3915
Epoch 12/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step - loss: 0.3762
Epoch 13/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 34s 34s/step - loss: 0.3698
Epoch 14/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - loss: 0.3572
Epoch 15/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - loss: 0.3348


In [32]:
bbb = model(X_train)

In [33]:
np.unique(bbb)

array([1.], dtype=float32)